In [16]:
import pandas as pd
import numpy as np

# Load CSV into a dataframe
og_df = pd.read_csv('loccalc for pain.xlsx - in.csv')

og_df.head()

,subject_id,stay_id,name,gsn,duration,intime,outtime,chiefcomplaint,duration (hours),log of avg duration,grade of medicine administered,grade of medicine administered.1,Level of Care
0,10018423.0,39057874.0,Aspirin,4380.0,0 days 11:05:00,29/4/2167,4/29/67 16:53,Chest pain,11.083333,0.923909,Aspirin,1.0,71.195458
1,10018423.0,33032677.0,Aspirin,4380.0,0 days 07:32:00,5/3/67 15:18,5/3/67 22:50,"Chest pain, Dizziness",7.533333,0.775610,Aspirin,1.0,63.780485
2,10014354.0,34617920.0,CefePIME,24095.0,0 days 20:39:00,5/8/50 22:30,5/9/50 19:09,"Abd pain, Dysuria",20.650000,1.162919,CefePIME,0.0,58.145933
3,10014354.0,34617920.0,CefePIME,24095.0,0 days 20:39:00,5/8/50 22:30,5/9/50 19:09,"Abd pain, Dysuria",20.650000,1.162919,CefePIME,0.0,58.145933
4,10014354.0,34617920.0,CefePIME,27470.0,0 days 20:39:00,5/8/50 22:30,5/9/50 19:09,"Abd pain, Dysuria",20.650000,1.162919,CefePIME,0.0,58.145933


In [ ]:
# removing grade of medicine and LOC and log avg duration
df = og_df.drop(columns=['grade of medicine administered.1', 'Level of Care', 'log of avg duration'])
df.head()

# filtering all chief complaints that contain the word "pain"
df = df[df['chiefcomplaint'].str.contains('pain', case=False, na=False)]

In [24]:
# noramlize duration column
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df['duration_normalized'] = scaler.fit_transform(df[['duration (hours)']])

In [25]:
print(max(df['duration_normalized']))
print(min(df['duration_normalized']))
print(np.average(df['duration_normalized']))
# (Testing to confirm if it works)

0.9999999999999998
0.0
0.1614650921733334


In [ ]:
# given that the values increase with intensity of GSN in comparison to the medicine administered, 
# creating a table with i.e., GSN390, aderal, GSN391, aderal => aderal1, aderal2 in a new df column

# Calculate GSN rank within each medicine group
df['gsn_rank'] = df.groupby('grade of medicine administered')['gsn'].rank(method='dense', ascending=False)
df['gsn_rank'] = df['gsn_rank'].fillna(0).astype(int)

# Create a combined rank column for each unique medicine-GSN pair
df['gsn_medicine_rank'] = df['grade of medicine administered'] + '_' + df['gsn_rank'].astype(str)

print(df[['gsn', 'grade of medicine administered', 'gsn_rank', 'gsn_medicine_rank']].head(10))

# finding frequency of each medicine-gsn pair
combo_counts = df['gsn_medicine_rank'].value_counts()
print(combo_counts)
# extract the frequency into a new column
df['gsn_medicine_freq'] = df['gsn_medicine_rank'].map(combo_counts)

       gsn grade of medicine administered  gsn_rank gsn_medicine_rank
0   4380.0                        Aspirin         1         Aspirin_1
1   4380.0                        Aspirin         1         Aspirin_1
2  24095.0                       CefePIME         2        CefePIME_2
3  24095.0                       CefePIME         2        CefePIME_2
4  27470.0                       CefePIME         1        CefePIME_1
5  27470.0                       CefePIME         1        CefePIME_1
6   3768.0                       Diazepam         1        Diazepam_1
7   4080.0                       Morphine         1        Morphine_1
8   4080.0                       Morphine         1        Morphine_1
9   4080.0                       Morphine         1        Morphine_1
gsn_medicine_rank
Morphine Sulfat 4mg/1mL 1mL VIAL_1    28
HYDROmorphone (D 1mg/1mL 1mL SYR_1    11
Morphine_1                            11
Ondansetron 4mg/2mL 2mL VIAL_2         9
Ondansetron 4mg/2mL 2mL VIAL_1         9
       

In [ ]:
# taking the last value of the gsn_medicine_rank (ie the rank)
df['gsn_medicine_rank_value']  = df['gsn_medicine_rank'].str.split('_').str[-1].astype(int)

# level of care calculation
df["level_of_care_encoded"] = df["duration_normalized"]*0.6 + og_df["grade of medicine administered.1"]*0.65 + df['gsn_medicine_rank_value']/10

In [54]:
print(max(df["level_of_care_encoded"]))
print(min(df["level_of_care_encoded"]))
print(np.average(df['level_of_care_encoded']))

2.0
0.012545955885686853
0.8005277039526487


In [1]:
df.to_csv('loccalc for pain - out.csv', index=False)

NameError: name 'df' is not defined